In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from transformers import MaskFormerFeatureExtractor, MaskFormerForInstanceSegmentation
import PIL
from PIL import Image
import requests
from transformers import pipeline
import datasets
from datasets import load_dataset
import os
import evaluate
import torch
import cv2
import json
import codecs

c:\Users\kuian\.conda\envs\master\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
image_folder = './WE3DS/images/'
annotation_folder = './WE3DS/annotations/segmentation/SegmentationLabel/'
# Define the paths to the images and annotations
all_image_names = np.array(os.listdir(image_folder))

In [3]:
plant_classification = {
    'void': 'void',
    'soil': 'soil',
    'broad bean': 'crop',
    'corn spurry': 'weed',
    'red-root amaranth': 'weed',
    'common buckwheat': 'crop',
    'pea': 'crop',
    'red fingergrass': 'weed',
    'common wild oat': 'weed',
    'cornflower': 'weed',
    'corn cockle': 'weed',
    'corn': 'crop',
    'milk thistle': 'weed',
    'rye brome': 'weed',
    'soybean': 'crop',
    'sunflower': 'crop',
    'narrow-leaved plantain': 'weed',
    'small-flower geranium': 'weed',
    'sugar beet': 'crop'
}

In [4]:
def get_image_meta_filepath(plant_name):
    return './meta/' + plant_name + '_images.json'

In [5]:
def get_image_list_for_crop(crop_name):
    # Create an empty list to store the dataset
    image_list = []
    crop_image_names = json.load(codecs.open(get_image_meta_filepath(crop_name), 'r', 'utf-8-sig'))
    weed_image_names = json.load(codecs.open(get_image_meta_filepath('no_crop'), 'r', 'utf-8-sig'))
    image_names = crop_image_names + weed_image_names

    # Iterate over the image and annotation paths
    for image_name in crop_image_names:
        # Load the image and annotation using PIL
        image = Image.open(image_folder + image_name)
        annotation = Image.open(annotation_folder + image_name)
        
        # Create a dictionary entry for the dataseta
        entry = {'image': image, 'annotation': annotation}
        
        # Add the entry to the dataset
        image_list.append(entry)

    return image_list

In [6]:
def create_and_split_dataset_for_crop(crop_image_list):
    dataset = datasets.Dataset.from_list(crop_image_list)
    dataset = dataset.train_test_split(test_size=0.5)
    train_ds = dataset["train"]
    val_ds, test_ds = dataset["test"].train_test_split(test_size=0.5).values()
    return train_ds, val_ds, test_ds

In [7]:
broad_bean_image_list = get_image_list_for_crop('broad_bean')
broad_bean_train_ds, broad_bean_val_ds, broad_bean_test_ds = create_and_split_dataset_for_crop(broad_bean_image_list)

In [8]:
print("Training subset number of images: " + str(broad_bean_train_ds.num_rows))
print("Validation subset number of images: " + str(broad_bean_val_ds.num_rows))
print("Test subset number of images: " + str(broad_bean_test_ds.num_rows))

Training subset number of images: 105
Validation subset number of images: 52
Test subset number of images: 53


In [9]:
from transformers import AutoImageProcessor

checkpoint = "nvidia/mit-b0"
image_processor = AutoImageProcessor.from_pretrained(checkpoint, reduce_labels=True)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
c:\Users\kuian\.conda\envs\master\lib\site-packages\transformers\models\segformer\image_processing_segformer.py:99: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


In [10]:
def train_transforms(example_batch):
    images = [x for x in example_batch["image"]]
    labels = [x for x in example_batch["annotation"]]
    inputs = image_processor(images, labels)
    return inputs

In [11]:
metric = evaluate.load("mean_iou")

In [12]:
with open('./WE3DS/class_names.txt', 'r') as file:
    labels = [line.strip() for line in file]

ids = list(range(1, 20))

id2label = dict(zip(ids, labels))
label2id = dict(zip(labels, ids))

num_labels = len(labels)

print("id2label:", id2label)
print("label2id:", label2id)

id2label: {1: 'void', 2: 'soil', 3: 'broad bean', 4: 'corn spurry', 5: 'red-root amaranth', 6: 'common buckwheat', 7: 'pea', 8: 'red fingergrass', 9: 'common wild oat', 10: 'cornflower', 11: 'corn cockle', 12: 'corn', 13: 'milk thistle', 14: 'rye brome', 15: 'soybean', 16: 'sunflower', 17: 'narrow-leaved plantain', 18: 'small-flower geranium', 19: 'sugar beet'}
label2id: {'void': 1, 'soil': 2, 'broad bean': 3, 'corn spurry': 4, 'red-root amaranth': 5, 'common buckwheat': 6, 'pea': 7, 'red fingergrass': 8, 'common wild oat': 9, 'cornflower': 10, 'corn cockle': 11, 'corn': 12, 'milk thistle': 13, 'rye brome': 14, 'soybean': 15, 'sunflower': 16, 'narrow-leaved plantain': 17, 'small-flower geranium': 18, 'sugar beet': 19}


In [13]:
def compute_metrics(eval_pred):
    with torch.no_grad():
        logits, labels = eval_pred
        logits_tensor = torch.from_numpy(logits)
        logits_tensor = torch.nn.functional.interpolate(
            logits_tensor,
            size=labels.shape[-2:],
            mode="bilinear",
            align_corners=False,
        ).argmax(dim=1)

        pred_labels = logits_tensor.detach().cpu().numpy()
        metrics = metric.compute(
            predictions=pred_labels,
            references=labels,
            num_labels=num_labels,
            ignore_index=255,
            reduce_labels=False,
        )
        for key, value in metrics.items():
            if type(value) is np.ndarray:
                metrics[key] = value.tolist()
        return metrics

In [14]:
from transformers import AutoModelForSemanticSegmentation, TrainingArguments, Trainer, EarlyStoppingCallback

model = AutoModelForSemanticSegmentation.from_pretrained(checkpoint, id2label=id2label, label2id=label2id)

Some weights of the model checkpoint at nvidia/mit-b0 were not used when initializing SegformerForSemanticSegmentation: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing SegformerForSemanticSegmentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SegformerForSemanticSegmentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.batch_norm.weight', 'decode_head.linear_c.2.proj.weight', 'decode_head.linear_c.1.proj.weight', 'decode_head.classifier.weight', 'decode_head.linear_c.2.pr

In [15]:
training_args = TrainingArguments(
    output_dir="segformer-b0-scene-parse-150",
    learning_rate=6e-5,
    num_train_epochs=10,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    save_total_limit=3,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=20,
    eval_steps=20,
    logging_steps=1,
    eval_accumulation_steps=5,
    remove_unused_columns=False,
    load_best_model_at_end=True,
)

In [16]:
def initialize_trainer(train_ds, test_ds) :
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=test_ds,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )
    return trainer

In [17]:
broad_bean_train_ds.set_transform(train_transforms)
broad_bean_val_ds.set_transform(train_transforms)
broad_bean_test_ds.set_transform(train_transforms)

In [18]:
braod_bean_trainer = initialize_trainer(broad_bean_train_ds, broad_bean_val_ds)
braod_bean_trainer.train()

c:\Users\kuian\.conda\envs\master\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  1%|          | 1/110 [00:17<30:55, 17.03s/it]

{'loss': 3.0092, 'learning_rate': 5.945454545454546e-05, 'epoch': 0.09}


  2%|▏         | 2/110 [00:33<29:49, 16.57s/it]

{'loss': 2.9758, 'learning_rate': 5.890909090909091e-05, 'epoch': 0.18}


  3%|▎         | 3/110 [00:49<29:22, 16.47s/it]

...

 99%|█████████▉| 109/110 [42:21<00:19, 19.84s/it]

{'loss': 1.8384, 'learning_rate': 5.454545454545455e-07, 'epoch': 9.91}


100%|██████████| 110/110 [42:30<00:00, 23.19s/it]

{'loss': 2.063, 'learning_rate': 0.0, 'epoch': 10.0}
{'train_runtime': 2550.7498, 'train_samples_per_second': 0.412, 'train_steps_per_second': 0.043, 'train_loss': 2.189501702785492, 'epoch': 10.0}


TrainOutput(global_step=110, training_loss=2.189501702785492, metrics={'train_runtime': 2550.7498, 'train_samples_per_second': 0.412, 'train_steps_per_second': 0.043, 'train_loss': 2.189501702785492, 'epoch': 10.0})

In [27]:
braod_bean_trainer.state.log_history

[{'loss': 3.0092,
  'learning_rate': 5.945454545454546e-05,
  'epoch': 0.09,
  'step': 1},
 {'loss': 2.9758,
  'learning_rate': 5.890909090909091e-05,
  'epoch': 0.18,
  'step': 2},
 {'loss': 2.9369,
  'learning_rate': 5.836363636363637e-05,
  'epoch': 0.27,
  'step': 3},
 {'loss': 2.9338,
  'learning_rate': 5.781818181818182e-05,
  'epoch': 0.36,
  'step': 4},
 {'loss': 2.8678,
  'learning_rate': 5.7272727272727274e-05,
  'epoch': 0.45,
  'step': 5},
 {'loss': 2.8533,
  'learning_rate': 5.6727272727272726e-05,
  'epoch': 0.55,
  'step': 6},
 {'loss': 2.8616,
  'learning_rate': 5.6181818181818184e-05,
  'epoch': 0.64,
  'step': 7},
 {'loss': 2.8397,
  'learning_rate': 5.5636363636363636e-05,
  'epoch': 0.73,
  'step': 8},
 {'loss': 2.7624,
  'learning_rate': 5.5090909090909094e-05,
  'epoch': 0.82,
  'step': 9},
 {'loss': 2.7696,
  'learning_rate': 5.4545454545454546e-05,
  'epoch': 0.91,
  'step': 10},
 {'loss': 2.7335,
  'learning_rate': 5.4000000000000005e-05,
  'epoch': 1.0,
  'ste

In [1]:
braod_bean_trainer.save_model('models/broad_bean')
# broad_bean_test_ds

NameError: name 'braod_bean_trainer' is not defined

In [20]:
# Interpret results
# Add method to rewrite labels in the annotions